# CS3481 Assignment 3 - Hierarchical clustering & K-means

This project uses the [UCI wine dataset](https://archive.ics.uci.edu/ml/datasets/Wine), which is imported using the built-in dataset of ```sklearn```. 

In [ ]:
import numpy
import sklearn
from scipy.cluster.hierarchy import dendrogram, fcluster, single, complete, average, linkage
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt 

In [ ]:
wine_dataset = load_wine()

# print(wine_data.feature_names)

_X = wine_dataset.data
_y = wine_dataset.target
print(_y.shape)
print(_X.shape)

## Data preprocessing
(1) Outliers disposal  (not implemented)  
(2) Normalization of some attributes - all attributes scaled to the same range

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(copy=True, feature_range=(10, 50))
X_scaler.fit(_X)
print("Per attribute max: ", X_scaler.data_max_)
print("Per attribute min: ", X_scaler.data_min_)

# X : normalized data matrix
X = X_scaler.transform(_X)


In [ ]:
# _X is the un-normalized version
plt.scatter(x=_X[:, 0], y=_X[:, 12])
plt.xlabel = "13th attribute"
plt.ylabel("1st attribute")
plt.show()
# X is the normalized version
plt.scatter(x=X[:, 0], y=X[:,12])
plt.xlabel = "13th attribute (normalized)"
plt.ylabel = "1st attribute (normalized)"
plt.show()

## Task 1
+ Try out three hierarchical clustering algorithms (single linkage, complete linkage, group average)
+ Visualize the dendrograms of the three clustering algorithms and compare their structures

In [ ]:
# single linkage clustering
Z_sig = linkage(X, method='single', metric='euclidean')
# complete linkage clustering
Z_cmp = linkage(X, method='complete', metric='euclidean')
# group average clustering
Z_avg = linkage(X, method='average', metric='euclidean')


In [ ]:
plt.figure(figsize = (25, 10))
dendrogram(Z_sig)
plt.show()

plt.figure(figsize = (25, 10))
dendrogram(Z_cmp)
plt.show()

plt.figure(figsize = (25, 10))
dendrogram(Z_avg)
plt.show()


## Task 2
Analyze the patterns in the clustering distance sequence of all three algorithms. Specifically, look at the "big gaps" in those sequences, since they usually mean a breakthrough in the cluster structures. 

In [ ]:
# find the distance sequence of single linkage
sig_dist = Z_sig[:, 2]
print(sig_dist.shape)

# visualize this sequence as a line chart 
fig, ax = plt.subplots()
ax.plot(numpy.arange(sig_dist.shape[0]), sig_dist, linewidth=2.0)
ax.set_title("Distance vs. no. of iteration for single linkage")
ax.set_xlabel("No. of iterations")
ax.set_ylabel("Distance")
plt.show()

In [ ]:
# inspect point X at index ~ 152
print(numpy.column_stack((numpy.arange(167, 177)[:, numpy.newaxis], Z_sig[167:177, :])))

In [ ]:
# find the distance sequence of complete linkage
cmp_dist = Z_cmp[:, 2]
print(cmp_dist.shape)

# visualize this sequence as a line chart 
fig, ax = plt.subplots()
ax.plot(numpy.arange(cmp_dist.shape[0]), cmp_dist, linewidth=2.0)
ax.set_title("Distance vs. no. of iteration for complete linkage")
ax.set_xlabel("No. of iterations")
ax.set_ylabel("Distance")
plt.show()

In [ ]:
# inspect point X at index ~ 152
print(numpy.column_stack((numpy.arange(160, 177)[:, numpy.newaxis], Z_cmp[160:177, :])))

In [ ]:
# find the distance sequence of group average
avg_dist = Z_avg[:, 2]
print(avg_dist.shape)

# visualize this sequence as a line chart 
fig, ax = plt.subplots()
ax.plot(numpy.arange(avg_dist.shape[0]), avg_dist, linewidth=2.0)
ax.set_title("Distance vs. no. of iteration for group average")
ax.set_xlabel("No. of iterations")
ax.set_ylabel("Distance")
plt.show()

In [ ]:
# inspect point X at index ~ 152
print(numpy.column_stack((numpy.arange(167, 177)[:, numpy.newaxis], Z_avg[167:177, :])))
# this index gets involved in merging
print("avg index 116:", Z_avg[116, :])

## Task 3
As we already know the number of class labels is 3 in the wine dataset, we want to make a comparison between the three hierarchical clustering algorithms (max. number of cluster = 3) and the K-means algorithm (K=3), in terms of how close their clustering schemes are to the ground truth.  

In [ ]:
# single linkage
sig_clusters_sol = fcluster(Z_sig, t=3, criterion='maxclust')
# complete linkage
cmp_clusters_sol = fcluster(Z_cmp, t=3, criterion='maxclust')
# group average 
avg_clusters_sol = fcluster(Z_avg, t=3, criterion='maxclust')


In [ ]:
# K-means with K=3
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=2323).fit(X)

In [ ]:
# kmeans.labels_.shape
print(sig_clusters_sol[170:178])
print(cmp_clusters_sol[170:178])
print(avg_clusters_sol[170:178])
print(kmeans.labels_[170:178])

In [ ]:
numpy.min(sig_clusters_sol)

A utility function to summarize the distribution of three ground truth classes in each of the clusters which are generated by the algorithm.

Output data structure of ```count_classes```:  

```
[
  [<count of class1>, <count of class 2>, <count of class 3>],  <--- for predicted class 0
  [<count of class1>, <count of class 2>, <count of class 3>],  <--- for predicted class 1
  ...
]
```

In [ ]:
def count_classes(y, cluster_sol, k=0):
  '''X: {ndarray} the original data matrix
     y: {ndarray} the ground truth labels
     cluster_sol: {ndarray} the clustering result
     k: {int}  how many kinds of clusters in cluster_sol
     Note: assume by default only 3 ground-truth classes
  '''
  # get total # of cluster labels (num_pred_clust) from cluster_sol
  num_pred_clust = numpy.max(cluster_sol) - numpy.min(cluster_sol) + 1
  print("# of predicted clusters: %d" % num_pred_clust)

  # if the labelling does not start with 0, then shift it to 0
  if (numpy.min(cluster_sol) > 0): 
    cluster_sol = cluster_sol - numpy.min(cluster_sol)

  # initialize a 2D nparray shape:(num_pred_clust, 3) with # of rows = total # of clusters 
  bin = numpy.zeros((num_pred_clust, 3))

  # for each row, there are three columns, corresponds to count of ground truth classes
  # <count of class 1>, <count of class 2>, <count of class 3>

  # iterate through all data objects 
  # for each object:
  # (1) get its predicted cluster label
  # (2) get its ground truth class label
  # (3) add 1 to 2D matrix[predicted cluster label, ground truth class label]
  for i in range(cluster_sol.shape[0]):
    pred_label = cluster_sol[i]
    gnd_truth_label = y[i]
    bin[pred_label, gnd_truth_label] +=1
  # outside the loop: return the 2D matrix
  return bin

def calc_impurity(bin):
  '''
     bin: the 2D matrix as output of count_classes()
     @return {ndarray} of shape (cnt_matrix.shape[0],), in which each row is the
       impurity measure of that particular predicted class
  '''
  print(bin)
  impurity_array = numpy.zeros(bin.shape[0], dtype=numpy.float32)
  for i in range(bin.shape[0]): # iterate through each row
    sub_total = numpy.sum(bin, axis=1)[i]
    entropy = 0
    for j in range(bin.shape[1]): # iterate though columns
      freq = bin[i, j] * 1.0 / sub_total 
      if freq == 0: # in entropy, log(0) is 0  
        pass
      else:
        entropy += (-1.0) * freq * numpy.log2(freq)
    impurity_array[i] = entropy
  return impurity_array 

In [ ]:
print(calc_impurity(count_classes(_y, sig_clusters_sol)))

In [ ]:
print(calc_impurity(count_classes(_y, cmp_clusters_sol)))

In [ ]:
print(calc_impurity(count_classes(_y, avg_clusters_sol)))

In [ ]:
print(calc_impurity(count_classes(_y, kmeans.labels_)))

## Task 4

Perform feature subset selection. Report if this can make the cluster structure more "balanced".

In [ ]:
len(wine_dataset.feature_names)

Try only select 9/6/3 features out of 13 features.

In [ ]:
from sklearn.utils.random import sample_without_replacement
# when n_samples = 9
# attr_list = sample_without_replacement(n_population=13, n_samples=9)
# attr_list.sort()

# whn n_samples = 6
# attr_list = sample_without_replacement(n_population=13, n_samples=6)
# attr_list.sort()
# when n_samples = 3
attr_list = sample_without_replacement(n_population=13, n_samples=3)
attr_list.sort()

Visualize cluster structures through dendrograms.

In [ ]:
X_sampled = X[:, attr_list[0]]
for i in range(1, 3):
  X_sampled = numpy.column_stack((X_sampled, X[:, attr_list[i]]))
print(X_sampled.shape)
# single linkage clustering
Z_sampled_sig = linkage(X_sampled, method='single', metric='euclidean')
# # complete linkage clustering
Z_sampled_cmp = linkage(X_sampled, method='complete', metric='euclidean')
 # group average clustering
Z_sampled_avg = linkage(X_sampled, method='average', metric='euclidean')

plt.figure(figsize = (25, 10))
dendrogram(Z_sampled_sig)
plt.show()

plt.figure(figsize = (25, 10))
dendrogram(Z_sampled_cmp)
plt.show()

plt.figure(figsize = (25, 10))
dendrogram(Z_sampled_avg)
plt.show()
